In [15]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from torch.utils.data import Dataset, DataLoader
import os
import matplotlib.pyplot as plt
import csv

import torch
import captum
from captum.attr import IntegratedGradients
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim


# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

device = "cpu"

Using device: cuda


In [16]:
# import argparse


# parser = argparse.ArgumentParser(description="Run script with optional CU and DU values.")
# parser.add_argument("--cu", type=int, default=0, help="CU value (default: 0)")
# parser.add_argument("--du", type=int, default=0, help="DU value (default: 0)")
# parser.add_argument("--lb", type=int, default=60, help="lookback value (default: 60)")
# parser.add_argument("--lf", type=int, default=1, help="lookforward value (default: 5)")

# args = parser.parse_args()

# CU, DU = args.cu, args.du
# LOOK_BACK, LOOK_FORWARD = args.lb, args.lf

CU = 0
DU = 0

LOOK_BACK, LOOK_FORWARD = 60, 1

In [17]:
# Load dataset
dataset = pd.read_csv(f'dataset_srscu{CU}_srsdu{DU}_pca.csv')
# dataset = dataset[:int(0.01*len(dataset))]

dataset.index = dataset['Timestamp']
dataset = dataset.drop(columns=['Timestamp'])

# dataset.head()

In [18]:
def create_sequences(df, look_back, look_forward, CU, DU):
    """
    Create LSTM-ready sequences from a DataFrame.

    Parameters:
        df (pd.DataFrame): Input dataframe.
        look_back (int): Number of timesteps to look back.
        look_forward (int): Number of timesteps to predict forward.

    Returns:
        X (np.ndarray): Shape (samples, look_back, features)
        y_df (pd.DataFrame): Shape (samples, features), from last step of forecast horizon
        column_names (list): List of column names used
    """
    print(df.columns)
    data = df.drop(columns=[f"srscu{CU}_stepStress", f"srscu{CU}_stressType", f"srsdu{DU}_stepStress", f"srsdu{DU}_stressType"]).values
    targets = df[[f"srscu{CU}_stepStress", f"srscu{CU}_stressType", f"srsdu{DU}_stepStress", f"srsdu{DU}_stressType"]].values

    pca_features_input, pca_features_output, input_targets, output_targets  = [], [], [], []

    for i in range(len(data) - look_back - look_forward + 1):
        pca_features_input.append(data[i:(i + look_back)])
        pca_features_output.append(data[i + look_back + look_forward - 1])  # last step of forecast

        input_targets.append(targets[i:(i + look_back)])
        output_targets.append(targets[i + look_back + look_forward - 1])  # last step of forecast

    return pca_features_input, pca_features_output, input_targets, output_targets





# **Define LSTM Model**
class LSTMForecaster(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTMForecaster, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

    def forward(self, x):
        h_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device)
        c_0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(device)
        out, _ = self.lstm(x, (h_0, c_0))
        out = self.fc(out[:, -1, :])
        return out


In [19]:
pca_features_input , pca_features_output, input_targets, output_targets = create_sequences(dataset, LOOK_BACK, LOOK_FORWARD, CU, DU)

print(f"Input shape: {np.array(pca_features_input).shape}")
print(f"Output shape: {np.array(pca_features_output).shape}")


Index(['Reduced-1', 'Reduced-2', 'Reduced-3', 'Reduced-4', 'Reduced-5',
       'Reduced-6', 'Reduced-7', 'Reduced-8', 'Reduced-9', 'srscu0_stepStress',
       'srscu0_stressType', 'srsdu0_stepStress', 'srsdu0_stressType'],
      dtype='object')
Input shape: (1439, 60, 9)
Output shape: (1439, 9)


In [ ]:
from sklearn.model_selection import KFold
import torch
from torch.utils.data import Subset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Convert to tensors for PyTorch
LSTM_input_tensor = torch.tensor(np.array(pca_features_input), dtype=torch.float32)  # (num_samples, 60, M)
LSTM_output_tensor = torch.tensor(np.array(pca_features_output), dtype=torch.float32)  # (num_samples, M)

save_dir="lstm_models"
os.makedirs(save_dir, exist_ok=True)


M = LSTM_input_tensor.shape[2]  # number of features per timestep, here it's 9

k_folds = 5
num_epochs = 5
batch_size = 64
learning_rate = 0.001

dataset = TensorDataset(LSTM_input_tensor, LSTM_output_tensor)
kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)

optimizers_to_try = {
    "Adam": lambda params: torch.optim.Adam(params, lr=learning_rate),
    "SGD": lambda params: torch.optim.SGD(params, lr=learning_rate, momentum=0.9),
    "RMSprop": lambda params: torch.optim.RMSprop(params, lr=learning_rate)
}

optimizers_to_try = {
    "RMSprop": lambda params: torch.optim.RMSprop(params, lr=learning_rate)
}

for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
    print(f"\n--- Fold {fold+1} ---")

    train_subset = Subset(dataset, train_idx)
    val_subset = Subset(dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False)

    for opt_name, opt_func in optimizers_to_try.items():
        print(f"\nTraining with optimizer: {opt_name}")


        model = LSTMForecaster(input_dim=M, hidden_dim=64, num_layers=2, output_dim=M).to(device)
        criterion = torch.nn.MSELoss()
        optimizer = opt_func(model.parameters())
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True, min_lr=1e-6)
        
        best_val_loss = float('inf')
        best_epoch = -1

        for epoch in range(num_epochs):
            model.train()
            train_loss = 0
            for batch_x, batch_y in train_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)

                optimizer.zero_grad()
                output = model(batch_x)
                loss = criterion(output, batch_y)
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * batch_x.size(0)

            avg_train_loss = train_loss / len(train_loader.dataset)

            model.eval()
            val_loss = 0
            with torch.no_grad():
                for val_x, val_y in val_loader:
                    val_x, val_y = val_x.to(device), val_y.to(device)
                    val_output = model(val_x)
                    loss_val = criterion(val_output, val_y)
                    val_loss += loss_val.item() * val_x.size(0)

            avg_val_loss = val_loss / len(val_loader.dataset)

            print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")

            scheduler.step(avg_val_loss)

            model_path = os.path.join(save_dir, f"lstm_fold{fold+1}_{opt_name}.pt")
            # Save best model for this optimizer/fold
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                best_epoch = epoch + 1
                torch.save(model, model_path)

        print(f"Best Val Loss for {opt_name} in Fold {fold+1}: {best_val_loss:.4f} at epoch {best_epoch}")



--- Fold 1 ---

Training with optimizer: RMSprop


/home/michael/miniconda3/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1: Train Loss = 0.2395, Val Loss = 0.2016
Epoch 2: Train Loss = 0.2161, Val Loss = 0.1929
Epoch 3: Train Loss = 0.2106, Val Loss = 0.1889
Epoch 4: Train Loss = 0.2063, Val Loss = 0.1860
Epoch 5: Train Loss = 0.2025, Val Loss = 0.1839
Best Val Loss for RMSprop in Fold 1: 0.1839 at epoch 5

--- Fold 2 ---

Training with optimizer: RMSprop
Epoch 1: Train Loss = 0.2319, Val Loss = 0.2417
Epoch 2: Train Loss = 0.2110, Val Loss = 0.2208
Epoch 3: Train Loss = 0.2024, Val Loss = 0.2156
Epoch 4: Train Loss = 0.1985, Val Loss = 0.2123
Epoch 5: Train Loss = 0.1940, Val Loss = 0.2074
Best Val Loss for RMSprop in Fold 2: 0.2074 at epoch 5

--- Fold 3 ---

Training with optimizer: RMSprop
Epoch 1: Train Loss = 0.2370, Val Loss = 0.2206
Epoch 2: Train Loss = 0.2137, Val Loss = 0.2084
Epoch 3: Train Loss = 0.2066, Val Loss = 0.2051
Epoch 4: Train Loss = 0.2031, Val Loss = 0.2001
Epoch 5: Train Loss = 0.1991, Val Loss = 0.1990
Best Val Loss for RMSprop in Fold 3: 0.1990 at epoch 5

--- Fold 4 ---